# Pull Polygons from TerraMatch API

This notebook sets up the process to pull polygon geometries and metadata from the TerraMatch API. The steps for pulling polygons are as follows:
1. Set up configuration and API token
2. 

In [18]:
import requests
import yaml
import json
import pandas as pd
import geopandas as gpd
from tqdm import tqdm
from datetime import datetime, timedelta

## Set up token and API URL

In [2]:
# Set up token access
auth_path = '../secrets.yaml'
with open(auth_path) as auth_file:
    auth = yaml.safe_load(auth_file)
headers = {
    'Authorization': f"Bearer {auth['access_token']}"
    }

In [3]:
# TerraMatch API URLs
staging_url = "https://api-staging.terramatch.org/research/v3/sitePolygons?" # Use for testing queries
prod_url = "https://api.terramatch.org/research/v3/sitePolygons?" # Use to pull data for analysis

## API Request

In [4]:
def pull_tm_api_data(url, headers, project_ids):
    '''
    edits to the above function include:
        iterating through list of project ids within func so output is a df with 
        multiple projects
        add project id as a column to support maxar metadata request
        update to last record variable
        added tqdm progress bar  
    '''
    # List to store all retrieved polygon metadata
    results = []
    # Set up a progress bar
    with tqdm(total=len(project_ids), desc="Processing Projects", unit="project") as progress_bar:
        # For every project in the list of project_ids
        for project_id in project_ids:
            # Set parameters with the current project ID
            params = {
                'projectId[]': project_id,
                'polygonStatus[]': 'approved',
                'includeTestProjects': 'false',
                'page[size]': '100'
            }

            last_record = ''
            new_last_record = None  # Ensure it's defined before use

            while True:
                # Send GET request and store the response (polygon geometries & metadata)
                response = requests.get(url, headers=headers, params=params)

                # Check status code
                if response.status_code != 200:
                    raise ValueError(f'Request failed for project {project_id} with status code {response.status_code}')
                
                # Convert the response to a JSON and record the total number of records returned
                response_json = response.json()
                total_records = response_json['meta']['page']['total']

                # Parse response data
                # If there are no polygons for this project
                if total_records == 0:
                    break  # Exit if no data is available (skip to the next project)

                # Loop through each polygon in the response
                for idx in range(total_records):
                    # Extract polygon attributes from each record and store them in dictionary data
                    data = response_json['data'][idx]['attributes']
                    data['poly_id'] = response_json['data'][idx]['meta']['page']['cursor']
                    # Store the project_id in data
                    data['project_id'] = project_id 
                    # Append data ( a dictionary of that project's metadata) in the overall results list
                    results.append(data)

                    # Assign the last cursor only if there are records
                    if idx == (total_records - 1):
                        new_last_record = response_json['data'][idx]['meta']['page']['cursor']

                # Check if there are more pages
                if new_last_record and last_record != new_last_record:
                    last_record = new_last_record
                    params['page[after]'] = last_record
                else:
                    break  # Exit pagination if no new cursor is found

            progress_bar.update(1) 
    return results

In [5]:
# Create the list of projects to pull
full = pd.read_csv('../projects_all_approved_202501091214.csv')
full = full[(full.framework_key == 'terrafund-landscapes') | (full.framework_key == 'terrafund')]
full.framework_key.value_counts()

framework_key
terrafund               108
terrafund-landscapes     99
Name: count, dtype: int64

In [6]:
full

,project_id,project_name,organization_name,organisation_id,country,framework_key,description,status
0,c462918b-47f7-4ed5-99e0-7fec6e342036,"""Nakuru Eco-Reforestation Project""",NaN,1382,KE,terrafund-landscapes,MAIN ACTIVITIES\nJAN - FEB 2024- TREE NURSERY...,approved
1,caae56f9-0bb6-45a2-9d77-ff088b085917,0726 project,NaN,6283,BR,terrafund,org 0726,approved
2,c004619e-c1aa-4f7f-b56b-c8f9b4385d4e,1,NaN,1582,AL,terrafund,1,approved
4,6083e1cf-a636-4c64-9253-ac86cd08f5d7,3SC Production 2.3,NaN,3279,AF,terrafund,3SC Production 2.3,approved
6,617601e0-9839-49fd-b48e-6c07404e7140,Afram Headwaters Restoration Initiative (AHRI),NaN,1358,GH,terrafund-landscapes,1.\tSite Reconnaissance – which has already be...,approved
...,...,...,...,...,...,...,...,...
268,2d1cffcd-300f-4939-a136-310347cf6879,WITHDRAWN: Divine Bamboo Afri 100,NaN,953,UG,terrafund,NOTE: THIS PROJECT HAS WITHDRAWN FROM TERRAFUN...,approved
269,568dc331-b945-41fd-ab57-80a98be57941,Women and Youth-Led Climate Initiative for res...,NaN,1814,KE,terrafund-landscapes,2024\n1. Community sensitization and mobilizat...,approved
270,77fc2c03-5605-45b3-b417-f69d93157215,Women led Community forest Conservation Init...,NaN,2815,KE,terrafund-landscapes,Project Activity \tSummary\n1. Stakeholders in...,approved
271,96c86eae-d4f9-45d8-9780-69c55a9e36e9,World Vision Ethiopia One Tree Planted Project,NaN,1047,ET,terrafund,07/01/2022 to 07/31/2023,approved


In [8]:
# Create a list of project ids to query
ids = list(set(full.project_id))

# Short list for testing purposes
ids = ids[:11]

In [9]:
# Pull polygons from projects in list of ids from TerraMatch API
project_results = pull_tm_api_data(prod_url, headers, ids)

Processing Projects:   0%|          | 0/11 [00:00<?, ?project/s]

Processing Projects: 100%|██████████| 11/11 [00:47<00:00,  4.27s/project]


In [10]:
# Convert the polygon geometries into a dataframe
project_df = pd.DataFrame(project_results)
project_df.columns = project_df.columns.str.lower()

In [11]:
project_df

,name,status,siteid,geometry,plantstart,plantend,practice,targetsys,distr,numtrees,calcarea,indicators,establishmenttreespecies,reportingperiods,poly_id,project_id
0,Nkoma 2 2024B,approved,59816e84-9bae-49c2-a493-1a91ec6e2781,"{'type': 'Polygon', 'coordinates': [[[30.48381...",2024-04-12,2024-04-12,tree-planting,agroforest,full,NaN,109.051824,"[{'indicatorSlug': 'restorationByStrategy', 'y...",[],"[{'dueAt': '2023-01-31T00:00:00.000Z', 'submit...",df3febb2-e506-4d6b-bf9c-325a8a885874,f81c1422-025c-45b1-a2e1-d354177523ca
1,Nkondo 1 Village 22A (new),approved,3c4f5af3-299a-4049-b0a6-57ac11662447,"{'type': 'Polygon', 'coordinates': [[[30.61055...",2022-04-11,None,tree-planting,agroforest,Null,NaN,11.235562,"[{'indicatorSlug': 'restorationByStrategy', 'y...",[],"[{'dueAt': '2022-09-30T00:00:00.000Z', 'submit...",729db629-99bb-4974-b8e2-4b67a65ac3f2,f81c1422-025c-45b1-a2e1-d354177523ca
2,Seka Village 22A (new),approved,3c4f5af3-299a-4049-b0a6-57ac11662447,"{'type': 'Polygon', 'coordinates': [[[30.60018...",2022-04-11,None,tree-planting,agroforest,Null,NaN,43.791715,"[{'indicatorSlug': 'restorationByStrategy', 'y...",[],"[{'dueAt': '2022-09-30T00:00:00.000Z', 'submit...",d81199c0-ad08-4440-b271-78bac24717ba,f81c1422-025c-45b1-a2e1-d354177523ca
3,Gasabo Village 22A (new),approved,3c4f5af3-299a-4049-b0a6-57ac11662447,"{'type': 'Polygon', 'coordinates': [[[30.61394...",2022-07-11,None,tree-planting,agroforest,Null,NaN,54.700154,"[{'indicatorSlug': 'restorationByStrategy', 'y...",[],"[{'dueAt': '2022-09-30T00:00:00.000Z', 'submit...",cd974393-b849-4fc6-831f-45dec0b849b0,f81c1422-025c-45b1-a2e1-d354177523ca
4,Burigade Village 22A (new),approved,3c4f5af3-299a-4049-b0a6-57ac11662447,"{'type': 'Polygon', 'coordinates': [[[30.60319...",2022-04-11,None,tree-planting,agroforest,Null,NaN,47.181002,"[{'indicatorSlug': 'restorationByStrategy', 'y...",[],"[{'dueAt': '2022-09-30T00:00:00.000Z', 'submit...",81deb295-8125-43e8-ac0e-01dd78c7fbc8,f81c1422-025c-45b1-a2e1-d354177523ca
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
263,"SAVE KENYA WATER TOWERS, MOROB SUB-LOCATION SITE",approved,ae5a9efd-66b0-4985-8c4c-7e733fa9363e,"{'type': 'Polygon', 'coordinates': [[[35.58452...",2024-05-01,2024-09-05,"tree-planting, assisted-natural-regeneration",agroforest,partial,45802.0,70.061969,"[{'indicatorSlug': 'restorationByStrategy', 'y...",[],"[{'dueAt': '2024-07-30T00:00:00.000Z', 'submit...",a91435c7-a179-4c1d-9891-de0fe1741654,146b6912-62a1-4b58-b027-466dc3295731
264,OESR Feature 2,approved,f3eac758-bb50-43b7-9cab-a8a32ed4b24a,"{'type': 'Polygon', 'coordinates': [[[0.446018...",2023-01-14,2023-09-29,tree-planting,natural-forest,full,68000.0,70.538672,"[{'indicatorSlug': 'restorationByStrategy', 'y...",[],"[{'dueAt': '2022-09-30T00:00:00.000Z', 'submit...",e41e8d8a-efa3-4626-bbfe-5af48f23b6da,529e1bae-2187-473f-a2a3-17e577720aba
265,OESR Feature 1,approved,f3eac758-bb50-43b7-9cab-a8a32ed4b24a,"{'type': 'Polygon', 'coordinates': [[[0.446577...",2022-10-14,2022-10-16,tree-planting,agroforest,full,2000.0,1.491618,"[{'indicatorSlug': 'restorationByStrategy', 'y...",[],"[{'dueAt': '2022-09-30T00:00:00.000Z', 'submit...",0b35ba48-a92c-48da-83d8-c21ddcdc3c0f,529e1bae-2187-473f-a2a3-17e577720aba
266,Site de Plantation 1,approved,0e58f16e-26a9-43d3-b5cc-bd4346a82273,"{'type': 'Polygon', 'coordinates': [[[12.33224...",2022-08-03,None,tree-planting,woodlot-or-plantation,full,0.0,21.003873,"[{'indicatorSlug': 'restorationByStrategy', 'y...",[],"[{'dueAt': '2022-09-30T00:00:00.000Z', 'submit...",727108b4-c899-45fc-b25b-1bbd5171782b,246a0906-7e03-4b57-bf71-b697b333509f


In [12]:
len(project_df.columns)

16

In [20]:
# Export the polygon geometries & metadata as a csv
today = datetime.today().strftime('%Y-%m-%d')
project_df.to_csv(f"../data/tm_api_{today}.csv", index=False) # To the darby-tm-api-pull repo
project_df.to_csv(f"/home/darby/github_repos/maxar-tools/data/tm_api_{today}.csv", index=False) # To the darby-maxar-tools repo